# An example of mastrplan creation

In [1]:
import pandas as pd
import networkx as nx
import geopandas as gpd

from masterplan_tools.City_model.city_model import CityModel
from masterplan_tools.Provision_getter.provision_getter import ProvisionModel
from masterplan_tools.Balancer.balancer import MasterPlan
from masterplan_tools.Data_getter.data_getter import DataGetter

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


#### City model creation

In [2]:
# To assess services provision they must be specified as a list of strings
# Service names must be the same as in your data

service_types = ["schools", "kindergartens"]

In [3]:
# path to data
path = "../mp_data/"

# load data required for blocks creation
city_geometry = gpd.read_parquet(path + 'city_geometry.parquet')
water_geometry = gpd.read_parquet(path + 'water_geometry.parquet')
roads_geometry = gpd.read_parquet(path + 'roads_geometry.parquet')
railways_geometry = gpd.read_parquet(path + 'railways_geometry.parquet')
nature_geometry_boundaries = gpd.read_parquet(path + 'nature_geometry_boundaries.parquet')

# load data required for service graphs creation
schools = gpd.read_parquet(path + 'schools.parquet')
kindergartens = gpd.read_parquet(path + 'kindergartens.parquet')
# accessibility_matrix = pd.read_pickle(path + "accessibility_matrix.pkl")

# load data required for 
buildings = gpd.read_parquet(path + 'buildings.parquet')
greenings = gpd.read_parquet(path + 'greenings.parquet')
parkings = gpd.read_parquet(path + 'parkings.parquet')
transport_graph = nx.read_graphml(path + 'new_graph.graphml')

In [4]:
# services should be specified as a dictionary
services = {
    'schools': schools,
    'kindergartens': kindergartens
}

In [ ]:
# City data model creation
city_model = CityModel(
    services=services,
    city_geometry=city_geometry,
    water_geometry=water_geometry,
    roads_geometry=roads_geometry,
    railways_geometry=railways_geometry,
    nature_geometry_boundaries=nature_geometry_boundaries,
    # accessibility_matrix=accessibility_matrix,
    transport_graph=transport_graph,
    buildings=buildings,
    greenings=greenings,
    parkings=parkings
)


#### Calculating the provision of school and kindergarten services in blocks

In [10]:
for service_type in services.keys():    
    model = ProvisionModel(
        city_model=city_model,
        service_name=service_type
        )

    if service_type == 'schools':
        school = model.run()
        
    elif service_type == 'kindergartens':
        kindergarten = model.run()


количество кварталов c сервисом schools: 0
количество жилых кварталов: 62
количество кварталов всего: 99
количество кварталов c ошибкой: 0
количество кварталов c сервисом kindergartens: 0
количество жилых кварталов: 39
количество кварталов всего: 49
количество кварталов c ошибкой: 0


In [21]:
school.explore()

#### Results of calculating the provision of the kindergarten service for a single block

In [12]:
kindergarten.iloc[15]

index                                                                             15
id                                                                                15
geometry                           POLYGON ((345325.11471557105 6648797.444055451...
provision_kindergartens                                                            0
id_kindergartens                                                                   0
population_prov_kindergartens                                                      0
population_unprov_kindergartens                                                    0
population                                                                         0
Name: 15, dtype: object

#### Results of calculating the provision of the school service for a single block

In [8]:
school.iloc[15]

index                                                                     6013
id                                                                        6013
geometry                     POLYGON ((365716.43716688757 6624359.046022324...
provision_schools                                                          100
id_schools                                                                6013
population_prov_schools                                                  14267
population_unprov_schools                                                    0
population                                                               14267
Name: 6013, dtype: object

#### Selected area of the city to which the new parameters will be modelled

In [9]:
polygon = gpd.read_file('../masterplanning/masterplan_tools/output_data/polygon.geojson')
polygon.explore()

#### Modelling of new parameters for the selected area with maximisation of the number of inhabitants

In [ ]:
block = DataGetter().balance_data(gdf=city_model.blocks_aggregated_info, polygon=polygon, school=school, kindergarten=kindergarten, greening=greening)
mp = MasterPlan(area=block['area'],
                current_living_area=block['current_living_area'],
                current_industrial_area=block['current_industrial_area'],
                current_population=block['current_population'],
                current_green_area=block['current_green_area'],
                current_unprov_schoolkids=block['population_unprov_schools'],
                current_unprov_kids=block['population_unprov_kindergartens'],
                current_unprov_green_population=block['population_unprov_recreational_areas'])

updated_block = mp.optimal_solution_indicators()


#### Modelled block transformation requirements

In [11]:
updated_block['block_id'] = 15
updated_block

{'area': 35.22888712300307,
 'population': 18454.0,
 'b': 30.0,
 'green_coef_G': 5.999999999999999,
 'living_area': 16.211970700000002,
 'schools_area': 1.3,
 'schools_capacity': 600.0,
 'kindergartens_area': 0.0,
 'kindergartens_capacity': 0.0,
 'green_area': 2.5122,
 'G_min_capacity': 4187.0,
 'G_max_capacity': 2093.5,
 'green_coef_G_capacity': 4187.0,
 'op_area': 0.012561000000000001,
 'parking1_area': 3.165372,
 'parking2_area': 3.077445}

#### Making changes based on requirements in the data model

In [ ]:
service_graphs = {}
for service_type in city_model.service_types:
   service_graphs[service_type] = DataGetter().prepare_graph(
                    blocks=city_model.city_blocks,
                    service_type=service_type,
                    buildings=city_model.buildings,
                    service_gdf=city_model.services_gdfs[service_type],
                    updated_block_info=updated_block,
                    accessibility_matrix=city_model.accessibility_matrix,
                )

In [25]:
city_model.services_graphs = service_graphs

#### Recalculating the provision of services to blocks

In [27]:
for service_type in city_model.service_types:    
    model = ProvisionModel(
    service_name=service_type,
    city_model=city_model
    )

    if service_type == 'schools':
        school = model.run()
        
    elif service_type == 'kindergartens':
        kindergarten = model.run()

количество кварталов c сервисом schools: 29
количество жилых кварталов: 1273
количество кварталов всего: 1609
количество кварталов c ошибкой: 0
{'population': 18454.0, 'is_living': True, 'is_schools_service': 1, 'schools_capacity': 3000, 'provision_schools': 0, 'id_schools': 0, 'population_prov_schools': 0, 'population_unprov_schools': 18454.0}
количество кварталов c сервисом kindergartens: 11
количество жилых кварталов: 797
количество кварталов всего: 900
количество кварталов c ошибкой: 0
{'population': 18454.0, 'is_living': True, 'is_kindergartens_service': 1, 'kindergartens_capacity': 743, 'provision_kindergartens': 0, 'id_kindergartens': 0, 'population_prov_kindergartens': 0, 'population_unprov_kindergartens': 18454.0}


#### Modified data on the availability of specified services

In [31]:
school.iloc[15]

index                                                                     6013
id                                                                        6013
geometry                     POLYGON ((365716.43716688757 6624359.046022324...
provision_schools                                                          100
id_schools                                                                6013
population_prov_schools                                                  18454
population_unprov_schools                                                    0
population                                                               18454
Name: 6013, dtype: object

In [28]:
kindergarten.iloc[15]

index                                                                           6013
id                                                                              6013
geometry                           POLYGON ((365716.43716688757 6624359.046022324...
provision_kindergartens                                                           66
id_kindergartens                                                                6013
population_prov_kindergartens                                                  12180
population_unprov_kindergartens                                                 6273
population                                                                     18454
Name: 6013, dtype: object